# Lab 5: 웨어하우스와 컨텍스트

👉 - 이 실습에서는 가상 웨어하우스에 대해 더 자세히 논의하고, 웨어하우스의 크기(컴퓨팅 용량)을 조정하는 옵션을 알아보고, 비용 정보를 확인할 수 있는 Snowsight의 옵션도 살펴봅니다.

먼저 이 실습 전반에 걸쳐 사용할 **컨텍스트 정보**를 가져오갰습니다. 

- **Start** 버튼을 클릭하여 이 노트북을 활성화합니다.

- 아래 Python 셀을 실행합니다.

#### :warning: 이 노트북에서 새 세션이 시작될 때마다, 이후 셀에서 사용할 "변수"를 설정하려면 아레 셀을 다시 실행해야 합니다. :warning:

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()
user = session.get_current_user().strip('"')
your_db = user + '_DB'
print('Your current CONTEXT information:')
print('---------------------------------')
print(session)
print('Your current USER is ' + user)

## Snowflake에서 웨어하우스란? 📓 

### Snowflake에서 "웨어하우스" 정의:

- 데이터 작업 경험이 있는 사람들은 "데이터 웨어하우스"라는 용어를 데이터 구조의 특별한 집합으로 생각할 수 있지만은, Snowflake에서는 웨어하우스가 데이터를 저장하지 않습니다.

- Snowflake에서 웨어하우스는 "컴퓨팅 리소스"입니다. 데이터를 처리하는 데 사용됩니다. 

- Snowflake에서 웨어하우스를 생성할 때, 이러한 "리소스"를 정의하는 것입니다.

### Scaling Up(수직 확장)과 Down(수직 축소): 

- 웨어하우스의 크기를 변경하면 클러스터 내의 서버 수가 변경됩니다. 

- 기존 웨어하우스의 크기를 변경하는 것을 Scaling Up(수직 확장) 또는 Scaling Down(수직 축소)이라고 합니다.

### Scaling In(수평 축소)와 Out(수평 확장): 

- 멀티 클러스터/탄력적 웨어하우징이 가능하다면(Enterprise 에디션 이상), 웨어하우스는 수요 증가 시 확장이 가능합니다.

- 멀티 클러스터 확장이 이루어질 경우, 수요 기간 동안 클러스터가 추가되고, 수요 감소 시 클러스터가 제거(snap back)됩니다.

- 웨어하우스가 클러스터를 추가하여 확장할 때 새로 추가되는 각 클러스터의 서버 수는 원래 클러스터 내 서버 수와 같습니다.

## 단지 가능하다고 해서... 📓 

### ...꼭 해야 한다는 의미는 아닙니다!!! 📓 

Snowflake에서는 몇 초만에 엄청난 컴퓨팅 파워를 사용할 수 있습니다! 특히 큰 작업을 위해 **큰** 컴퓨팅 파워가 필요한 경우 가능하다는 점을 알아두세요.

하지만 대부분의 쿼리는 대규모 컴퓨팅 파워를 필요로 **하지 않는다**는 점도 알아두세요.

사실, Snowflake는 항상 Extra-Small(XS) 웨어하우스에서 시작하고, 정말 필요한 경우에만 확장할 것을 권장합니다. XS 웨어하우스는 시간당 1 크레딧을 사용합니다. 가장 큰 웨어하우스는 Snowpark-optimized 6XL이며, 시간당 768 크레딧을 사용합니다! 이는 분당 38 크레딧 이상입니다!

이 워크숍에서, 웨어하우스 크기를 S로 요청하는 경우를 제외하고 XS로 설정하고 유지하세요.

실제 업무 환경에서 Snowflake를 사용할 때는, 웨어하우스 구성을 관리하는 사람이 있을 가능성이 큽니다. 웨어하우스를 과도한 크기로 설정하는 것은 월 청구서에서 큰 놀라움을 초래할 수 있는 가장 간단한 실수입니다. 따라서 대부분의 경우 XS와 S 웨어하우스를 사용하고, 신중히 고려한 후에만 확장하는 것이 좋습니다.

Snowflake는 각 계정에 비용을 관리하는 사람을 두는 것을 권장합니다. 이들은 웨어하우스 크기를 선택하고 탄력적 설정을 구성하는 방법에 대해 고급 지식을 보유해야 합니다. 이러한 비용 관리자는 웨어하우스 크기 변경이 발생 비용을 정당화할 만큼의 시간 절감을 가져오는지, 또는 비용을 균형 있게 조정할 수 있는지 계산할 수 있어야 할수도 있습니다.

## 가상 웨어하우스 크기 조정 🥋

Snowflake 가상 웨어하우스 크기는 Snowsight 또는 프로그래밍 방식으로 조정할 수 있습니다. 지정된 가상 웨어하우스를 이용해서 이러한 옵션을 알아보겠습니다.

### Snowsight를 사용하여 가상 웨어하우스를 "확장(Scale Up)"하기

Snowsight 웨어하우스 페이지를 새 브라우저 또는 탭으로 엽니다.

1. 노트북 왼쪽의 메인 메뉴에서 **Admin** 아이콘 위에 마우스 커서를 올립니다.

1. 나타나는 대화 상자의 **Warehouses** 링크를 마우스 오른쪽 버튼으로 클릭하고, 선호하는 대로 새 탭 또는 새 창으로 열기를 선택합니다.

1. 이렇게 하면 현재 Snowflake 노트북 페이지를 방해하지 않고 유지할 수 있으며, 쉽게 다시 돌아와 지침을 확인할 수 있습니다.

![Open Warehouses page (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_admin_menu_1.png)

1. 동물 이름이 붙은 웨어하우스 라인을 찾습니다.

1. 해당 라인의 오른쪽에 있는 줄임표(점 3개)를 클릭하고 **Edit** 옵션을 선택합니다.

![Open warehouses edit dialog (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_warehouse_edit_1.png)

1. **Size** 드롭다운을 클릭합니다.

1. **Small 2 credits/hour** 옵션을 선택합니다.

1. 파란색 **Save Warehouse** 버튼을 클릭합니다.

![Adjust warehouse size (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_warehouse_edit_2.png)

### 코드를 사용하여 가상 웨어하우스 크기 확인하기 🥋

`SHOW` 명령어를 사용하면 가상 웨어하우스의 현재 크기를 포함한 여러 세부 정보를 확인할 수 있습니다.

아래 명령어를 실행하고, 웨어하우스의 "size" 컬럼의 값을 확인하세요:

In [ ]:
SHOW warehouses like '{{user}}_wh';

### 코드를 사용하여 가상 웨어하우스 크기 조정하기 🥋

가상 웨어하우스에 대해 이러한 작업을 수행할 수 있는 권한이 있다면, `ALTER` 구문을 사용하여 여러 속성을 변경하거나 실행 중인 쿼리를 취소하고, 웨어하우스를 일시 중단(suspend, 정지) 하거나 재개(resume, 시작)할 수 있습니다.

위 실습에서 Snowsight를 사용하여 확장한 웨어하우스를 이전 크기로 되돌리기 위해 `ALTER` 구문을 실행하세요.

In [ ]:
ALTER warehouse {{user}}_wh SET warehouse_size=XSMALL;

### `SHOW` 명령어를 사용하여 웨어하우스 크기 변경이 성공적으로 이루어졌는지 확인하세요. 🎯 

- 아래 SQL 셀에 정보(현재 크기를 포함)를 표시하는 `SHOW` 명령어를 **작성**하고 실행하세요.

💡 **힌트**: 아래 셀에는 코드가 제공되지 않았지만, 이전에 이 노트북에서 실행한 명령어를 참고하여 작성할 수 있습니다.

In [ ]:
SHOW warehouses like '{{user}}_wh';

### 예상치 못한 상황으로부터 보호하기 📓 

Snowflake는 누군가가 실수를 하더라도 가능한 한 빨리 이를 인지할 수 있도록 비용을 모니터링하고 제어하는 다양한 방법을 제공합니다. 일반적으로 플랫폼 관리자가 Snowflake 계정에서 이러한 옵션을 구성하고 관리합니다. 몇 가지 옵션은 다음과 같습니다:

- [Budgets](https://docs.snowflake.com/ko/user-guide/budgets)
- [Resource Monitors](https://docs.snowflake.com/ko/user-guide/resource-monitors)
- [Alerts and Notifications](https://docs.snowflake.com/ko/guides-overview-alerts)


### 비용 정보 조회 📓

Snowflake는 플랫폼에서의 지출을 추적할 수 있도록 여러 기능과 도구를 제공합니다. **SNOWFLAKE** 데이터베이스의 **ACCOUNT_USAGE** 및 **ORGANIZATION_USAGE** 스키마의 뷰에 대해 쿼리를 작성하여 이 정보를 얻을 수 있습니다. 더욱 간단하게는 Snowsight를 사용해 과거 비용을 탐색할 수도 있습니다. Snowsight는 시각적 대시보드를 통해 비용 정보를 빠르고 쉽게 확인할 수 있게 합니다.

기본적으로, 비용 및 사용 데이터에 접근할 수 있는 권한은 계정 관리자(`ACCOUNTADMIN` 역할을 가진 사용자)에게만 부여됩니다. 하지만 이 계정의 `(animal)_LEARNER_RL` 역할에도 이러한 권한이 부여되어 있습니다.

계정 수준 비용을 확인하려면:
- Snowsight의 왼쪽 메인 메뉴에서 **Admin** » **Cost Management**를 선택해서 새 브라우저 탭 또는 창에서 이를 엽니다.

![Access cost management (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_cost_management_link_1.png)

### Account Overview

- 화면이 뜨면, 사용량 데이터를 확인하기 위해 사용할 웨어하우스를 선택합니다. Snowflake는 이 용도로 X-Small 웨어하우스 사용을 권장합니다.
- **Account Overview** 페이지는 Snowflake 사용 비용에 대한 개괄적인 인사이트를 제공하며, 비용 최적화를 시작하는 출발점이 될 수 있습니다.

![Account Overview (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_account_overview_ui.png)

### Consumption

- **Consumption** 페이지를 사용하면 특정 일, 주 또는 월에 대해 Snowflake 사용 전체 비용을 상세히 확인할 수 있습니다.

![Account Overview (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_consumption_ui.png)


**참고**: 이 페이지에는 다양한 하위 메뉴와 클릭가능한 구성 요소가 포함되어 있어, 요구사항에 따라 비용을 세부적으로 확인, 필터, 정렬할 수 있습니다.

## Lesson 5 챌린지 실습 🎯 

### 계정 수준 비용 탐색

#### Cost Management > Account Overview 페이지 🎯 

![Account Overview (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_account_overview_ui_small.png)

Snowsight에서 이 페이지를 직접 탐색하고 다음 질문에 답해보세요:

- 지난 28일 동안 이 계정의 **average daily credits**은 얼마입니까?
- 가장 큰 데이터베이스는 무엇입니까?
- 이 기간 동안 가장 비용이 많이 든 쿼리를 실행한 사용자는 누구입니까?

#### Cost Management > Consumption 페이지 🎯 

![Account Overview (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_consumption_ui_small.png)

Snowsight에서 이 페이지를 직접 탐색하고 다음 질문에 답해보세요:

- 지난 28일 동안 가장 많은 크레딧이 소비된 날은 언제입니까?
- 이 기간 동안 AI 서비스에서 크레딧을 소비한 적이 있습니다.
- 이 계정에서 데이터 저장소로 사용된 리전은 어디입니까?

## 컨텍스트 설정 📓 

**Does not exist** 오류는 종종 발생하며, 이는 항상 잘못된 역할을 사용한 탓만은 아닙니다.

종종 다음과 같은 다른 이유로 does not exist error 오류가 발생할 수 있습니다:

- **잘못된 위치**에 무엇인가를 **생성** - 예를들어 ROOT_DEPTH 테이블을 FRUITS 스키마에 생성한 경우

- **잘못된 위치를 조회**한 경우 -- 데이터베이스 컨텍스트가 `SNOWFLAKE_SAMPLE_DATA.PUBLIC`으로 설정된 상태에서 `SELECT * FROM ROOT_DEPTH;`를 실행한 경우

- **오타**가 있는 경우 -- `SELECT * FROM GARENPLNT.VEGGIES.ROOT_DEPTH;`와 같은 쿼리

물론, 해당 항목을 **생성하지 않았을** 가능성도 있습니다! 이 가능성도 확인해 보세요!

또한 지금쯤이면 다음 사항을 알고 있어야 합니다...

- USER(사용자)와 ROLE(역할)의 차이점 

- USER(사용자)와 ACCOUNT(계정)의 차이점

- DEFAULT ROLE(기본 역할)과 Snowsight 홈페이지의 CURRENT ROLE(현재 역할), 노트북에서 설정한 ROLE(역할) 간의 차이점

- Snowflake 노트북에서 프로그램적으로 역할을 변경하는 방법

위 사항들을 모르면 아래 질문을 해결하는 데 어려움을 겪을 수 있습니다. 만약 어려움이 있다면, 다시 수업을 복습해 보세요.

## 지식 테스트 🔎

아래 Python 셀을 실행하여 Streamlit 기반의 위젯을 표시하고 Snowflake 특징과 기능에 대한 질문에 답하세요. 지금은 이 코드가 무엇을 하는지 이해할 필요가 없습니다. 그냥 코드를 실행하세요.

질문에 올바르게 답해야 다음 섹션으로 진행할 수 있습니다.

In [ ]:
import streamlit as st
st.divider()
question = "Snowflake 웨어하우스에 대한 올바른 설명은 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) Snowflake 웨어하우스는 조직의 데이터 하위 섹션을 보유하는 데이터 마트와 같습니다.", 
           "B) 기능적으로, Snowflake 웨어하우스는 노트북 CPU보다는 스프레드시트 워크북과 비슷합니다.", 
           "C) Snowflake 웨어하우스는 쿼리를 실행하고, 데이터를 로드하며, 기타 작업을 수행하기 위한 컴퓨팅 파워를 제공합니다.",
           "D) Snowflake 웨어하우스는 Kimball 및 Inmon 데이터 웨어하우징 이론에 기반한 구조로 데이터를 저장하도록 설계되었습니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = 'edd8bbd8673536ec1a22174223040328'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "Scaling OUT/BACK과 Scaling UP/DOWN은 쉽게 혼동됩니다. Snowflake 웨어하우스 스케일링에 대한 올바른 설명은 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) XS 웨어하우스를 M으로 변경하는 것은 Scaling UP의 예입니다.", 
           "B) XS 웨어하우스를 M으로 변경하는 것은 Scaling OUT의 예입니다.", 
           "C) XS 웨어하우스가 증가된 작업 부하를 처리하기 위해 자동으로 클러스터를 추가하는 것은 Scaling Up의 예입니다.",
           "D) XS 웨어하우스가 증가된 작업 부하를 처리하기 위해 자동으로 클러스터를 추가하는 것은 Scaling Down의 예입니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '1cb4d62237ac722fefc975761c57ce4d'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "웨어하우스는 수동으로 UP 또는 DOWN으로 규모 조정할 수 있습니다. 또한 자동으로 Scale OUT하도록 설정할 수 있습니다. SCALING OUT의 반대는 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) De-Scaling", 
           "B) Scaling Back", 
           "C) Shedding Clusters",
           "D) Scaling DOWN"]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = 'd32360b1231ec658361bec6b720bd70b'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "웨어하우스 규모조정은 서버와 클러스터 모두와 관련이 있습니다. 웨어하우스 크기 조정 및 규모조정에서 서버와 클러스터의 역할에 대한 올바른 설명은 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) 서버는 단순히 클러스터 그룹을 의미합니다.", 
           "B) M 크기 웨어하우스(Scale Out되지 않은 경우)는 XS 크기 웨어하우스보다 더 많은 클러스터를 가지고 있습니다.", 
           "C) 서버는 여러 클러스터를 포함할 수 있습니다.",
           "D) 클러스터는 여러 서버를 포함할 수 있습니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...": 
        ''
    else:
        answer = '881f0353d1d0a52f0e35aa9306ce8133'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "기존 웨어하우스를 언제 Scale Up 해야 합니까?"
options = ["아래 보기 중 고르세요...",
           "A) 지루하고 뭔가 재미있는 것을 시도해보고 싶을 때", 
           "B) Scaling Up의 영향을 신중히 고려했을 때: 시간당 사용되는 크레딧 증가", 
           "C) 자유분방한 동료가 보수적인 동료보다 더 큰 소리로 그렇게 하라고 외칠 때"]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...": 
        ''
    else:
        answer = '7adf4a847127fc42564abd23d6dbbebb'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

## 다음 단계

실습 단계를 완료하고 **지식 테스트** 질문에 올바르게 답했다면, Snowflake 강사의 안내에 따라 다음 노트북으로 진행하세요.